In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

#Load the trained model, scalar, pickle and onehot encoder

In [2]:
model = load_model('model.h5')

## load the encoder and scalar

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('onehot_encoder_location.pkl','rb') as file:
    onehot_encoder_location = pickle.load(file)
    
with open('scalar.pkl','rb') as file:
    scalar = pickle.load(file)
    
    

In [4]:
#example input data

input_data = {
    'CreditScore':600,
    'Location' : 'California',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCreditCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [ ]:
# one hot encode location
location_encoded = onehot_encoder_location.transform([[input_data['Location']]]).toarray()
location_encoded_df = pd.DataFrame(location_encoded, columns = onehot_encoder_location.get_feature_names_out(['Location']))

location_encoded_df

c:\Users\nagur\Desktop\prac\venv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Location_California,Location_Florida,Location_Illinois,Location_New York,Location_Texas
0,1.0,0.0,0.0,0.0,0.0


In [12]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Location,Gender,Age,Tenure,Balance,NumOfProducts,HasCreditCard,IsActiveMember,EstimatedSalary
0,600,California,Male,40,3,60000,2,1,1,50000


In [13]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Location,Gender,Age,Tenure,Balance,NumOfProducts,HasCreditCard,IsActiveMember,EstimatedSalary
0,600,California,1,40,3,60000,2,1,1,50000


In [14]:
input_df = pd.concat([input_df.drop('Location',axis=1), location_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCreditCard,IsActiveMember,EstimatedSalary,Location_California,Location_Florida,Location_Illinois,Location_New York,Location_Texas
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0,0.0,0.0


In [15]:
##scaling the input data
input_scaled_data = scalar.transform(input_df)
input_scaled_data

array([[ 0.17613679,  1.02020406, -0.64695271, -0.62136509, -0.89359004,
        -0.46706375,  0.996257  ,  1.02532046, -0.99535285,  2.03585229,
        -0.49706857, -0.49119477, -0.50974751, -0.51072038]])

In [17]:
##prediction
prediction=model.predict(input_scaled_data)
prediction

1/1 [==============================] - 0s 373ms/step


array([[0.35998008]], dtype=float32)

In [19]:
prediction_probability = prediction[0][0]
prediction_probability

0.35998008

In [20]:
if prediction_probability > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
